<img style="float: left;" alt="Drawing" src="./figures/3Di_beeldmerk_RGB.png" width="100"/>

## Starting a 3Di simulation with control structures

Welcome! In this notebook we will show you how to start a <a href="https://3diwatermanagement.com/">3Di</a>-simulation in a jupyter notebook by using the API-v3. In addition, we will show you how to set control structures.

The example we use sets the discharge coefficient in two orifices to almost zero based on a water level at a certain connection node. This is an example of the use of control structures. 3Di knows various control structures: timed control, table control, & memory control. For more detail check out the documentation: https://docs.3di.live/c_control.html .

The following steps will be taken according to an example of an 3Di model:
- **step 1:** Creating a 3Di-simulation by using the threedi-api
- **step 2:** Adding events to this 3Di-simulation including control structures by using the threedi-api
- **step 3:** Running the 3Di-simulation by using the threedi-api

**Step 1: Starting a 3Di-simulation by using the threedi-api**

Importing all required packages:

In [1]:
from datetime import datetime
from getpass import getpass
from threedi_api_client.threedi_api_client import ThreediApiClient
from threedi_api_client.api import ThreediApi
from threedi_api_client.versions import V3Api

from login import get_login_details
import configparser


Provide your credentials to connect to the threedi-api:

In [2]:
CONFIG = {
    "THREEDI_API_HOST": "https://api.3di.live",
    "THREEDI_API_USERNAME": get_login_details(option='username'),
    "THREEDI_API_PASSWORD": get_login_details(option='password')
}  #
THREEDI_API = ThreediApi(config=CONFIG, version='v3-beta')

Check the connection with your provided credentials:

In [3]:
try:
    user = THREEDI_API.auth_profile_list()
except ApiException as e:
    print("Oops, something went wrong. Maybe you made a typo?")
else:
    print(f"Successfully logged in as {user.username}!")

Successfully logged in as olof.baltus!


1.3  In order to run a simulation you need a threedi-model. Let's see which threedi-models are available:

In [4]:
part_of_the_model_name = "schermer"

models = THREEDI_API.threedimodels_list(name__icontains=part_of_the_model_name, limit = "20")  # limit to the first 20 results
for model in models.results:
    print(f"{model.name, model.id}")
    
#print(models)
   


('bwn-schermer - bwn_schermer_1d2d_glg (5) #20', 52244)
('RegionalFloodModel - deelmodel Schermer Midden Zuid #11', 52234)
('RegionalFloodModel - deelmodel Schermer Hoog Zuid #9', 52210)
('RegionalFloodModel - deelmodel Schermer Hoog Zuid #8', 52209)
('RegionalFloodModel - deelmodel Schermer Hoog Zuid #6', 52203)
('RegionalFloodModel - deelmodel Schermer Laag Noord #7', 52194)
('RegionalFloodModel - deelmodel Schermer Midden Noord #12', 52166)
('RegionalFloodModel - deelmodel Schermer Midden Noord #11', 52056)
('RegionalFloodModel - deelmodel Schermer Laag Noord #6', 51957)
('RegionalFloodModel - deelmodel Schermer Midden Noord #10', 51407)
('RegionalFloodModel - deelmodel Schermer Laag Noord #5', 51405)
('RegionalFloodModel - deelmodel Schermer Midden Zuid #10', 51119)
('RegionalFloodModel - deelmodel Schermer Hoog Noord #10', 51024)
('RegionalFloodModel - deelmodel Schermer Laag Zuid #9', 51010)
('RegionalFloodModel - deelmodel Schermer Hoog Noord #9', 50468)
('RegionalFloodModel - d

In [5]:
#set the model id of the model to use in this calculation
model_id = 50417

In this notebook we will use a sample model with control structures in it

Now that we have a model we are almost ready to create the simulation. However, first we'll need to get an organisation under which's name we will run the simulation.

Let's see which organisations are available within my user account:

In [6]:
organisations = THREEDI_API.organisations_list(name__icontains="N&S")

for organisation in organisations.results:
    print(f"{organisation.name}: {organisation.unique_id}")

N&S Branch Office: 74983d5a0fd84c22bb09e7656d570fb6
N&S Demo: a1993f6e13564e9687ae03a3604463f9


In this example we use the organisation from Nelen & Schuurmans Demo:

organisation_uuid = "a1993f6e13564e9687ae03a3604463f9"

In [7]:
organisation_uuid = "a1993f6e13564e9687ae03a3604463f9" #N&S Demo

1.5 Let's create the simulation of the chosen model now, with this organisation uuid. Note that it will not run yet.

In [8]:
name_for_simulation = "demo_simulation_control_structures"
sim_duration = 3600

my_simulation = THREEDI_API.simulations_create(
    data={
        "name": name_for_simulation,
        "threedimodel": model_id,
        "organisation": organisation_uuid,
        "start_datetime": datetime.now(),
        "duration": sim_duration  # in seconds, so we simulate for 1 hour
    }
)


#print an overview of the simulation
my_simulation

{'cloned_from': None,
 'compute_cluster': None,
 'created': 'now',
 'duration': 3600,
 'duration_humanized': '1 hours, 0 minutes, 0 seconds',
 'end_datetime': datetime.datetime(2022, 10, 24, 15, 59, 9, tzinfo=tzutc()),
 'id': 99307,
 'name': 'demo_simulation_control_structures',
 'organisation': 'a1993f6e13564e9687ae03a3604463f9',
 'organisation_name': 'N&S Demo',
 'slug': 'demo_simulation_control_structures-f3e91fe3-40ed-4499-9b00-0ff4c02b7465',
 'start_datetime': datetime.datetime(2022, 10, 24, 14, 59, 9, tzinfo=tzutc()),
 'tags': [],
 'threedicore_version': None,
 'threedimodel': 'https://api.3di.live/v3/threedimodels/50417/',
 'threedimodel_id': 50417,
 'url': 'https://api.3di.live/v3/simulations/99307/',
 'user': 'olof.baltus',
 'uuid': 'f3e91fe3-40ed-4499-9b00-0ff4c02b7465'}

You can check the status of the simulation with the following api call:

In [9]:
#check the status of the simulation with:
status = THREEDI_API.simulations_status_list(my_simulation.id)
print(status)

{'created': datetime.datetime(2022, 10, 24, 12, 58, 53, 557670, tzinfo=tzutc()),
 'exit_code': None,
 'id': 463120,
 'name': 'created',
 'paused': None,
 'time': 0.0}


We can see the simulation has not started yet. The options at the name of the status can be: "created", "started" and "finished".

**Step 2: Adding events to this 3Di-simulation by using the threedi-api**

In the previous step we created a simulation for the 3Di model of rockflow. Several events can be added to this 3Di-simulation:

* initial waterlevels
* rain
* breaches
* laterals
* control structures 


In this step we create a simple constant rain event of 30 minutes with an offset of 5 minutes and directly adding it to our simulation:

In [10]:
THREEDI_API.simulations_events_rain_constant_create(    
    simulation_pk=my_simulation.id,   # the ID we got from our create call above
    data={
        "offset": 300,       # let the rain start after 5 minutes
        "duration": 1800,    # let the rain last for half an hour
        "value": 0.0006,     # not too extreme after all...;-)
        "units": "m/s",      # the only unit supported for now
    },
)

{'duration': 1800,
 'id': 26518,
 'offset': 300,
 'simulation': 'https://api.3di.live/v3/simulations/99307/',
 'uid': 'b09b5d7e-254b-485b-894a-45884492ee18',
 'units': 'm/s',
 'url': 'https://api.3di.live/v3/simulations/99307/events/rain/constant/26518/',
 'value': 0.0006}

2.3 We can get an overview of the added events to our 3Di-simulation by the following api-call:

In [11]:
events = THREEDI_API.simulations_events(my_simulation.id)
print(events)

{'breach': [],
 'fileboundaryconditions': None,
 'filelaterals': [],
 'filerasterleakage': [],
 'filerasterrain': [],
 'filerastersourcessinks': [],
 'filestructurecontrols': [],
 'filetimeseriesleakage': [],
 'filetimeseriesrain': [],
 'filetimeseriessourcessinks': [],
 'initial_groundwaterlevel': None,
 'initial_groundwaterraster': None,
 'initial_onedwaterlevel': None,
 'initial_onedwaterlevelfile': None,
 'initial_onedwaterlevelpredefined': None,
 'initial_savedstate': None,
 'initial_twodwaterlevel': None,
 'initial_twodwaterraster': None,
 'initial_winddragcoefficient': None,
 'laterals': [],
 'leakage': [],
 'lizardrasterrain': [],
 'lizardrastersourcessinks': [],
 'lizardtimeseriesrain': [],
 'lizardtimeseriessourcessinks': [],
 'localrain': [],
 'memorystructurecontrols': [],
 'rasteredits': [],
 'savedstates': [],
 'tablestructurecontrols': [],
 'timedstructurecontrols': [],
 'timeseriesrain': [{'constant': True,
                     'duration': 1800,
                     'id

So, we can indeed see here that we have only added the constant rain event to our 3di-simulation.

**Step 2.2. Adding controls to the simulation**

In [12]:
#help(simulation_api) #use this line to see all options available
help(THREEDI_API.simulations_events_structure_control_table_create)  # we use this one the request information on the endpoint

Help on method simulations_events_structure_control_table_create in module threedi_api_client.openapi.api.v3_api:

simulations_events_structure_control_table_create(simulation_pk, data, **kwargs) method of threedi_api_client.versions.V3BetaApi instance
    simulations_events_structure_control_table_create  # noqa: E501
    
    A table structure control can used to apply a certain action, e.g increase the pump capacity of a pump station, based on the measured values of one or more locations.  **type**  *for weirs and orifices*  - set_crest_level - set_discharge_coefficients  *for culverts, channels and pipes*  - set_discharge_coefficients  *for pumps*  - set_pump_capacity (m3/s)  **values**  The action table consists of one or more value pairs (threshold, action_value)  To close/open or partially close/open a structure using the `set_discharge_coefficients` type the values must contain **three** values. For example `[[1.2, 0.5, 0.7]]`,  where     1.2 is the threshold value     0.5 the 

In [13]:
# As we can see we need to provide data on the control. 

from openapi_client.models import MeasureSpecification
from openapi_client.models import MeasureLocation
from openapi_client.models import TableStructureControl

C:\Users\olof.baltus\AppData\Roaming\Python\Python39\site-packages\openapi_client\__init__.py:22: UserWarning: openapi_client is pending deprecation. Please use the new threedi_api_client.ThreediApi and the models in threedi_api_client.openapi.
  warnings.warn(


In [14]:
# First we define the location where the measure is taken. 
measure_location1 = MeasureLocation (
    weight = 1.0,
    content_type = 'v2_connection_node',
    content_pk = 92776
)

# Then we define what is measured and the operator sign. This indicates whether the control has to act when (in this example waterlevel) is higher or lower than a certain value
measure1 = MeasureSpecification (
    name = 'measure point 1',
    locations = [measure_location1], #in case of multiple locations add as a list
    variable = 's1', #waterlevel measurement variable, one of the following options: s1 (waterlevel), vol1 (volume), q (discharge), u1 (velocity)
    operator = '>')

control_in_simulation = TableStructureControl (
    duration = sim_duration,
    offset = 0,
    measure_specification = measure1, #defined before
    structure_id = 5, #this is the id of the structure in the model. 
    structure_type ='v2_orifice', #[ v2_pumpstation, v2_pipe, v2_orifice, v2_culvert, v2_weir, v2_channel ]
    type = 'set_discharge_coefficients', # [ set_discharge_coefficients, set_crest_level, set_pump_capacity ]
    values = [[-2.60,0.0063,0.0063]] #When the water level exceeds the value of -2.60 m MSL the discharge coefficient of this structure is set to 0.063 (almost closed)
)

In [15]:
THREEDI_API.simulations_events_structure_control_table_create(my_simulation.id, control_in_simulation)

{'duration': 3600,
 'grid_id': None,
 'id': 4506,
 'measure_specification': {'id': 4511,
                           'locations': [{'content_pk': 92776,
                                          'content_type': 'v2_connection_node',
                                          'grid_id': None,
                                          'id': 4511,
                                          'state': 'processing',
                                          'state_detail': None,
                                          'weight': '1.00'}],
                           'name': 'measure point 1',
                           'operator': '>',
                           'variable': 's1'},
 'offset': 0,
 'state': 'processing',
 'state_detail': None,
 'structure_id': 5,
 'structure_type': 'v2_orifice',
 'type': 'set_discharge_coefficients',
 'uid': 'c20b88fe-27ba-4e40-85de-d18ff1bce0d6',
 'url': 'https://api.3di.live/v3/simulations/99307/events/structure-control/table/4506/',
 'values': [[-2.6, 0.0063, 0.

In [16]:
# First we define the location where the measure is taken. 
measure_location2 = MeasureLocation (
    weight = 1.0,
    content_type = 'v2_connection_node',
    content_pk = 92776
)

# Then we define what is measured and the operator sign. This indicates whether the control has to act when (in this example waterlevel) is higher or lower than a certain value
measure2 = MeasureSpecification (
    name = 'measure point 2',
    locations = [measure_location2], #in case of multiple locations add as a list
    variable = 's1', #waterlevel measurement variable, one of the following options: s1 (waterlevel), vol1 (volume), q (discharge), u1 (velocity)
    operator = '>')

control_in_simulation2 = TableStructureControl (
    duration = sim_duration,
    offset = 0,
    measure_specification = measure2, #defined before
    structure_id = 6, #this is the id of the structure in the model. 
    structure_type ='v2_orifice', #[ v2_pumpstation, v2_pipe, v2_orifice, v2_culvert, v2_weir, v2_channel ]
    type = 'set_discharge_coefficients', # [ set_discharge_coefficients, set_crest_level, set_pump_capacity ]
    values = [[-2.60,0.0063,0.0063]] #When the water level exceeds the value of -2.60 m MSL the discharge coefficient of this structure is set to 0.063 (almost closed)
)

In [17]:

THREEDI_API.simulations_events_structure_control_table_create(my_simulation.id, control_in_simulation2)

{'duration': 3600,
 'grid_id': None,
 'id': 4507,
 'measure_specification': {'id': 4512,
                           'locations': [{'content_pk': 92776,
                                          'content_type': 'v2_connection_node',
                                          'grid_id': None,
                                          'id': 4512,
                                          'state': 'processing',
                                          'state_detail': None,
                                          'weight': '1.00'}],
                           'name': 'measure point 2',
                           'operator': '>',
                           'variable': 's1'},
 'offset': 0,
 'state': 'processing',
 'state_detail': None,
 'structure_id': 6,
 'structure_type': 'v2_orifice',
 'type': 'set_discharge_coefficients',
 'uid': '09a81846-b799-430f-b448-71be55682066',
 'url': 'https://api.3di.live/v3/simulations/99307/events/structure-control/table/4507/',
 'values': [[-2.6, 0.0063, 0.

**Step 3: Running the 3Di-simulation by using the threedi-api**

We will now start our simulation with the constant rain event:

In [19]:
my_simulation_template = THREEDI_API.simulation_templates_list(simulation__threedimodel__id=model_id).results[0]
my_simulation = THREEDI_API.simulations_from_template(
        data={
            "template": my_simulation_template.id,
            "name": "demo_simulation_3dinotebook_schermer",
            "tags": ["demo_notebook_1"],
            "organisation": organisation_uuid,
            "start_datetime": datetime.now(),
            "duration": 3600 # in seconds, so we simulate for 1 hour
        }
    )

In [20]:
THREEDI_API.simulations_actions_create(my_simulation.id, data={"name": "start"})

{'compute_cluster': None,
 'duration': None,
 'max_rate': None,
 'name': 'start',
 'timeout': 300}

We can check the status of the 3Di-simulation with:

In [22]:
#check the status of the simulation with:
status = THREEDI_API.simulations_status_list(my_simulation.id)
print(status)

{'created': datetime.datetime(2022, 10, 24, 13, 1, 53, 582225, tzinfo=tzutc()),
 'exit_code': '1240 [finished]',
 'id': 463128,
 'name': 'finished',
 'paused': False,
 'time': 3612.0}



**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**
The end

In this tutorial we showed you:
- how to start a simulation of a 3Di-model, 
- how to add a simple rain event to your simulation, 
- how to add control to your simulation
- how to see the progress

And all possible within a Jupyter notebook by making use of the API-v3. Ofcourse there are a lot more options for the analysis possible, by using threedigrid. Or for the events you can add to your simulation. Do you want to learn more or are you interested in some more possible analysis in a jupyter notebook? 

**Please contact**:

Olof Baltus | olof.baltus@nelen-schuurmans.nl
Jonas van Schrojenstein | jonas.vanschrojenstein@nelen-schuurmans.nl
